# DETAILS

**Project Name:** Visa Jobs NL

**Description:** Notebook to collect, process, and analyze job listings related to visa sponsorship opportunities in the Netherlands.  

It performs the mounting of Azure Data Lake Storage containers for bronze, silver, and gold data layers,  
extracts raw job listing data, transforms it into clean and structured formats, and prepares the data for further analytics and visualization.

**Author(s):**  
+ Daniel Willians <daniel.wis@outlook.com>

**Start Date (YYYY-MM-DD):** 2025-05-27

**Version:** 1.0.0


# Assign Role to Service Principal in Azure Key Vault

Para conceder a permissão de **Key Vault Secrets User** a um Service Principal em um cofre de chaves (Key Vault), use o comando `az role assignment create` do Azure CLI.

## Comando utilizado

```bash
az role assignment create \\
  --assignee-object-id <OBJECT_ID_DO_SERVICE_PRINCIPAL> \\
  --assignee-principal-type ServicePrincipal \\
  --role "Key Vault Secrets User" \\
  --scope /subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.KeyVault/vaults/<KEYVAULT_NAME>
```
###Explicação
*assignee-object-id*: ID do Service Principal que receberá a permissão.

*assignee-principal-type*: Tipo do principal, aqui especificado como ServicePrincipal.

*role*: Nome da role a ser atribuída (neste caso, Key Vault Secrets User).

*scope*: Escopo onde a permissão será aplicada — o recurso do Key Vault.



In [0]:
# Configurações principais
storage_account = dbutils.secrets.get(scope="kv-estudo-prod-scope", key="sas-token-datalake-name") 
container = "netherlands-tracker"
storage_key = dbutils.secrets.get(scope="kv-estudo-prod-scope", key="sas-token-datalake-key") 

# Lista de camadas a montar
layers = ["bronze", "silver", "gold"]

for layer in layers:
    mount_point = f"/mnt/{container}/{layer}"
    source = f"wasbs://{container}@{storage_account}.blob.core.windows.net/{layer}"

    # Desmontar se já estiver montado
    if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        print(f"Unmounting existing mount at {mount_point}...")
        dbutils.fs.unmount(mount_point)

    # Montar
    try:
        print(f"Mounting {layer} layer at {mount_point}...")
        dbutils.fs.mount(
            source=source,
            mount_point=mount_point,
            extra_configs={
                f"fs.azure.account.key.{storage_account}.blob.core.windows.net": storage_key
            }
        )
        print(f"✅ {layer.capitalize()} mounted successfully at {mount_point}")
    except Exception as e:
        print(f"❌ Failed to mount {layer}: {e}")

In [0]:
for mount in dbutils.fs.mounts():
    if mount.mountPoint.startswith("/mnt/"):
        print(mount.mountPoint, "->", mount.source)